In [1]:
import spotipy
import pandas as pd
import numpy as np
import requests
import random
from spotipy.oauth2 import SpotifyClientCredentials
import time
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id="a3a2927271164d2786c28785f365ae9b",
    client_secret="5e240c22b70140ac8b613a2529b0ccd2"))

In [3]:
large_playlist = "spotify:playlist:7htu5ftbLBRFAwiuHVcUAg"

In [4]:
playlist = sp.user_playlist_tracks("spotify", large_playlist)

STRUCTURE

In [5]:
#List of artists
artists_list =set()
for item in playlist['items']:
    for artist in item ['track']['artists']:
        artists_list.add(artist['uri'])

In [6]:
# From  artist to album
albums =[]
for artist in artists_list:
    albums.append(sp.artist_albums(artist_id = artist))
    
albums_list = set()
for album in albums:
    for item in album['items']:
        albums_list.add(item['uri'])

In [7]:
songs =[]
for album in albums_list:
    songs.append(sp.album_tracks(album_id=album))

songs_list = []
for song in songs:
    for item in song['items']:
        songs_list.append(item['uri'])

In [8]:
len(songs_list)

12627

In [9]:
songs_name_list = []
for song in songs:
    for item in song['items']:
        songs_name_list.append(item['name'])

In [10]:
len(songs_name_list)

12627

In [11]:
artist_name_list = []
for song in songs:
    for item in song['items']:
        artist_name_list.append(item['artists'][0]['name'])

In [12]:
len(artist_name_list)

12627

Creating list and DF of all the song features

In [13]:
all_features = []
for songs in songs_list:
    all_features.append(sp.audio_features(songs))

In [14]:
len(all_features)

12627

In [46]:
all_features

[[{'danceability': 0.661,
   'energy': 0.601,
   'key': 3,
   'loudness': -5.448,
   'mode': 0,
   'speechiness': 0.0776,
   'acousticness': 0.0614,
   'instrumentalness': 1.06e-06,
   'liveness': 0.12,
   'valence': 0.369,
   'tempo': 127.713,
   'type': 'audio_features',
   'id': '63XN4O5GTc5moJ6ln9Jvbf',
   'uri': 'spotify:track:63XN4O5GTc5moJ6ln9Jvbf',
   'track_href': 'https://api.spotify.com/v1/tracks/63XN4O5GTc5moJ6ln9Jvbf',
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/63XN4O5GTc5moJ6ln9Jvbf',
   'duration_ms': 210600,
   'time_signature': 4}],
 [{'danceability': 0.666,
   'energy': 0.889,
   'key': 7,
   'loudness': -5.699,
   'mode': 1,
   'speechiness': 0.144,
   'acousticness': 0.0987,
   'instrumentalness': 0,
   'liveness': 0.0965,
   'valence': 0.916,
   'tempo': 149.959,
   'type': 'audio_features',
   'id': '1PruWB9LHLSn67U87JQ5Eo',
   'uri': 'spotify:track:1PruWB9LHLSn67U87JQ5Eo',
   'track_href': 'https://api.spotify.com/v1/tracks/1PruWB9LHLSn67U87JQ5

In [15]:
long = pd.DataFrame(all_features[0], columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'])

In [68]:
full = pd.DataFrame(all_features[0], columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'])

In [17]:
for i in range(1, len(all_features)):
    long = long.append(pd.DataFrame((all_features[i]), columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'uri', ]))

In [70]:
for i in range(1, len(all_features)):
    full = full.append(pd.DataFrame((all_features[i]), columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']))

In [71]:
full

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.661,0.601,3,-5.448,0,0.0776,0.06140,0.000001,0.1200,0.3690,127.713
0,0.666,0.889,7,-5.699,1,0.1440,0.09870,0.000000,0.0965,0.9160,149.959
0,0.549,0.414,7,-8.453,1,0.0273,0.14600,0.000000,0.0998,0.4040,149.746
0,0.610,0.667,11,-5.825,0,0.0314,0.00131,0.000180,0.3400,0.5100,96.954
0,0.635,0.718,0,-7.775,0,0.1190,0.03040,0.000383,0.2330,0.7870,171.915
...,...,...,...,...,...,...,...,...,...,...,...
0,0.413,0.374,0,-10.744,1,0.0321,0.43300,0.000000,0.1110,0.1050,144.930
0,0.541,0.556,7,-11.523,1,0.0255,0.38300,0.707000,0.1070,0.4730,110.046
0,0.171,0.564,6,-6.936,1,0.0312,0.05510,0.020500,0.1880,0.0618,133.993
0,0.508,0.807,11,-8.581,1,0.0686,0.33300,0.003630,0.1360,0.3060,123.034


ELiminate duplicates in the DF

In [48]:
final_song_data = long.drop_duplicates()

In [50]:
long

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.661,0.601,3,-5.448,0,0.0776,0.06140,0.000001,0.1200,0.3690,127.713
0,0.666,0.889,7,-5.699,1,0.1440,0.09870,0.000000,0.0965,0.9160,149.959
0,0.549,0.414,7,-8.453,1,0.0273,0.14600,0.000000,0.0998,0.4040,149.746
0,0.610,0.667,11,-5.825,0,0.0314,0.00131,0.000180,0.3400,0.5100,96.954
0,0.635,0.718,0,-7.775,0,0.1190,0.03040,0.000383,0.2330,0.7870,171.915
...,...,...,...,...,...,...,...,...,...,...,...
0,0.413,0.374,0,-10.744,1,0.0321,0.43300,0.000000,0.1110,0.1050,144.930
0,0.541,0.556,7,-11.523,1,0.0255,0.38300,0.707000,0.1070,0.4730,110.046
0,0.171,0.564,6,-6.936,1,0.0312,0.05510,0.020500,0.1880,0.0618,133.993
0,0.508,0.807,11,-8.581,1,0.0686,0.33300,0.003630,0.1360,0.3060,123.034


In [51]:
final_song_data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.661,0.601,3,-5.448,0,0.0776,0.061400,0.000001,0.1200,0.369,127.713
0,0.666,0.889,7,-5.699,1,0.1440,0.098700,0.000000,0.0965,0.916,149.959
0,0.549,0.414,7,-8.453,1,0.0273,0.146000,0.000000,0.0998,0.404,149.746
0,0.610,0.667,11,-5.825,0,0.0314,0.001310,0.000180,0.3400,0.510,96.954
0,0.635,0.718,0,-7.775,0,0.1190,0.030400,0.000383,0.2330,0.787,171.915
...,...,...,...,...,...,...,...,...,...,...,...
0,0.494,0.622,8,-13.459,0,0.5190,0.043200,0.000000,0.1810,0.662,107.996
0,0.661,0.520,2,-12.478,1,0.3580,0.087700,0.000000,0.3080,0.423,106.719
0,0.939,0.450,4,-7.928,0,0.2480,0.000353,0.000000,0.2940,0.357,98.970
0,0.423,0.659,4,-10.371,0,0.4000,0.412000,0.000000,0.6400,0.672,178.207


In [ ]:
final_full_data['song'] = songs_name_list

In [ ]:
final_full_data["artist"] = artist_name_list

In [20]:
final_full_data = full.drop_duplicates()

In [21]:
final_song_data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.661,0.601,3,-5.448,0,0.0776,0.061400,0.000001,0.1200,0.369,127.713
0,0.666,0.889,7,-5.699,1,0.1440,0.098700,0.000000,0.0965,0.916,149.959
0,0.549,0.414,7,-8.453,1,0.0273,0.146000,0.000000,0.0998,0.404,149.746
0,0.610,0.667,11,-5.825,0,0.0314,0.001310,0.000180,0.3400,0.510,96.954
0,0.635,0.718,0,-7.775,0,0.1190,0.030400,0.000383,0.2330,0.787,171.915
...,...,...,...,...,...,...,...,...,...,...,...
0,0.494,0.622,8,-13.459,0,0.5190,0.043200,0.000000,0.1810,0.662,107.996
0,0.661,0.520,2,-12.478,1,0.3580,0.087700,0.000000,0.3080,0.423,106.719
0,0.939,0.450,4,-7.928,0,0.2480,0.000353,0.000000,0.2940,0.357,98.970
0,0.423,0.659,4,-10.371,0,0.4000,0.412000,0.000000,0.6400,0.672,178.207


In [22]:
final_full_data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.661,0.601,3,-5.448,0,0.0776,0.061400,0.000001,0.1200,0.369,127.713
0,0.666,0.889,7,-5.699,1,0.1440,0.098700,0.000000,0.0965,0.916,149.959
0,0.549,0.414,7,-8.453,1,0.0273,0.146000,0.000000,0.0998,0.404,149.746
0,0.610,0.667,11,-5.825,0,0.0314,0.001310,0.000180,0.3400,0.510,96.954
0,0.635,0.718,0,-7.775,0,0.1190,0.030400,0.000383,0.2330,0.787,171.915
...,...,...,...,...,...,...,...,...,...,...,...
0,0.494,0.622,8,-13.459,0,0.5190,0.043200,0.000000,0.1810,0.662,107.996
0,0.661,0.520,2,-12.478,1,0.3580,0.087700,0.000000,0.3080,0.423,106.719
0,0.939,0.450,4,-7.928,0,0.2480,0.000353,0.000000,0.2940,0.357,98.970
0,0.423,0.659,4,-10.371,0,0.4000,0.412000,0.000000,0.6400,0.672,178.207


No duplicates in the DF

Initialize Scaling - Fit Transform 

In [52]:
long.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'],
      dtype='object')

In [24]:
scaler = StandardScaler()

In [25]:
scaler

StandardScaler()

In [53]:
f_s_d = scaler.fit_transform(long)

In [54]:
f_s_d.shape

(12627, 11)

In [55]:
import pickle
pickle.dump(scaler, open("scaler_final.pkl", "wb"))

In [56]:
data_scaled = pd.DataFrame(f_s_d, columns= long.columns)

CLUSTERING

Initialize Modelling

In [57]:
k_means = KMeans(n_clusters=20, random_state=1234)

Fit the model

In [58]:
data_scaled.shape

(12627, 11)

In [59]:
k_means_final = k_means.fit(data_scaled)

In [60]:
pickle.dump(k_means_final, open("KMeans_final.pkl", "wb"))

In [61]:
clusters = k_means.predict(data_scaled)

In [62]:
clusters

array([13,  4, 10, ..., 10,  2,  1])

In [63]:
pd.Series(clusters).value_counts()

19    974
18    910
2     901
13    856
9     856
6     800
4     768
10    760
16    760
5     665
15    616
17    592
0     554
1     531
11    479
14    437
8     406
3     331
12    230
7     201
dtype: int64

In [64]:
data_scaled["clusters"] = clusters

In [65]:
data_scaled

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,clusters
0,0.606438,0.056830,-0.591387,0.603681,-1.407937,-0.086826,-0.798421,-0.240282,-0.472183,-0.321971,0.334015,13
1,0.638780,1.343477,0.506726,0.540343,0.710259,0.368145,-0.684381,-0.240289,-0.587409,2.044976,1.086653,4
2,-0.118031,-0.778597,0.506726,-0.154601,0.710259,-0.431480,-0.539766,-0.240289,-0.571229,-0.170521,1.079446,10
3,0.276546,0.351686,1.604840,0.508549,-1.407937,-0.403387,-0.982140,-0.239072,0.606528,0.288156,-0.706639,9
4,0.438258,0.579530,-1.414972,0.016486,-1.407937,0.196846,-0.893200,-0.237698,0.081882,1.486774,1.829479,11
...,...,...,...,...,...,...,...,...,...,...,...,...
12622,-0.997742,-0.957298,-1.414972,-0.732711,0.710259,-0.398591,0.337705,-0.240289,-0.516312,-1.464337,0.916509,17
12623,-0.169778,-0.144209,0.506726,-0.929284,0.710259,-0.443814,0.184836,4.542331,-0.535925,0.128052,-0.263704,12
12624,-2.563112,-0.108469,0.232198,0.228199,0.710259,-0.404758,-0.817683,-0.101614,-0.138763,-1.651269,0.546483,10
12625,-0.383238,0.977140,1.604840,-0.186900,0.710259,-0.148494,0.031966,-0.215734,-0.393732,-0.594581,0.175713,2


In [72]:
full

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.661,0.601,3,-5.448,0,0.0776,0.06140,0.000001,0.1200,0.3690,127.713
0,0.666,0.889,7,-5.699,1,0.1440,0.09870,0.000000,0.0965,0.9160,149.959
0,0.549,0.414,7,-8.453,1,0.0273,0.14600,0.000000,0.0998,0.4040,149.746
0,0.610,0.667,11,-5.825,0,0.0314,0.00131,0.000180,0.3400,0.5100,96.954
0,0.635,0.718,0,-7.775,0,0.1190,0.03040,0.000383,0.2330,0.7870,171.915
...,...,...,...,...,...,...,...,...,...,...,...
0,0.413,0.374,0,-10.744,1,0.0321,0.43300,0.000000,0.1110,0.1050,144.930
0,0.541,0.556,7,-11.523,1,0.0255,0.38300,0.707000,0.1070,0.4730,110.046
0,0.171,0.564,6,-6.936,1,0.0312,0.05510,0.020500,0.1880,0.0618,133.993
0,0.508,0.807,11,-8.581,1,0.0686,0.33300,0.003630,0.1360,0.3060,123.034


In [73]:
full["clusters"] = clusters

In [74]:
full['uri'] = songs_list

In [75]:
full["song"] = songs_name_list

In [76]:
full["artist"] = artist_name_list

In [78]:
data_scaled.to_csv("../Labs/Data/Spotify_Data.csv")

In [79]:
full.to_csv("../Labs/Data/Full_data.csv")